# 01 Cointegration and Error Correction

Engle-Granger cointegration and error correction models (ECM).


## Table of Contents
- [Construct cointegrated pair](#construct-cointegrated-pair)
- [Engle-Granger test](#engle-granger-test)
- [Error correction model](#error-correction-model)
- [Interpretation](#interpretation)
- [Checkpoint (Self-Check)](#checkpoint-self-check)
- [Solutions (Reference)](#solutions-reference)


## Why This Notebook Matters
Time-series econometrics notebooks build the classical toolkit you need before trusting macro regressions:
- stationarity + unit roots,
- cointegration + error correction,
- VAR dynamics and impulse responses.


## Prerequisites (Quick Self-Check)
- Completed Part 01 macro panel notebooks (or have `panel_monthly.csv` / sample available).
- Comfort with differencing/log transforms and reading time series plots.

## What You Will Produce
- (no file output; learning/analysis notebook)

## Success Criteria
- You can explain what you built and why each step exists.
- You can run your work end-to-end without undefined variables.

## Common Pitfalls
- Running cells top-to-bottom without reading the instructions.
- Leaving `...` placeholders in code cells.
- Running tests without plotting or transforming the series first.
- Treating impulse responses as structural causality without an identification story.

## Quick Fixes (When You Get Stuck)
- If you see `ModuleNotFoundError`, re-run the bootstrap cell and restart the kernel; make sure `PROJECT_ROOT` is the repo root.
- If a `data/processed/*` file is missing, either run the matching build script (see guide) or use the notebook’s `data/sample/*` fallback.
- If results look “too good,” suspect leakage; re-check shifts, rolling windows, and time splits.
- If a model errors, check dtypes (`astype(float)`) and missingness (`dropna()` on required columns).

## Matching Guide
- `docs/guides/07_time_series_econ/01_cointegration_error_correction.md`



## How To Use This Notebook
- Work section-by-section; don’t skip the markdown.
- Most code cells are incomplete on purpose: replace TODOs and `...`, then run.
- After each section, write 2–4 sentences answering the interpretation prompts (what changed, why it matters).
- Prefer `data/processed/*` if you have built the real datasets; otherwise use the bundled `data/sample/*` fallbacks.
- Use the **Checkpoint (Self-Check)** section to catch mistakes early.
- Use **Solutions (Reference)** only to unblock yourself; then re-implement without looking.
- Use the matching guide (`docs/guides/07_time_series_econ/01_cointegration_error_correction.md`) for the math, assumptions, and deeper context.



<a id="environment-bootstrap"></a>
## Environment Bootstrap
Run this cell first. It makes the repo importable and defines common directories.



In [ ]:
from __future__ import annotations

from pathlib import Path
import sys


def find_repo_root(start: Path) -> Path:
    p = start
    for _ in range(8):
        if (p / 'src').exists() and (p / 'docs').exists():
            return p
        p = p.parent
    raise RuntimeError('Could not find repo root. Start Jupyter from the repo root.')


PROJECT_ROOT = find_repo_root(Path.cwd())
if str(PROJECT_ROOT) not in sys.path:
    sys.path.append(str(PROJECT_ROOT))

DATA_DIR = PROJECT_ROOT / 'data'
RAW_DIR = DATA_DIR / 'raw'
PROCESSED_DIR = DATA_DIR / 'processed'
SAMPLE_DIR = DATA_DIR / 'sample'

PROJECT_ROOT



## Goal
Learn cointegration and error correction models (ECM):
- long-run equilibrium relationship
- short-run dynamics that correct deviations



## Primer: Classical time-series econometrics with `statsmodels` (ADF/KPSS, cointegration, VAR/IRF)

This repo already teaches time-aware evaluation for ML. This primer introduces the classical econometrics toolkit for time series:
- stationarity / unit roots,
- cointegration and error correction ideas,
- VARs and impulse responses.

Deep theory is in the guides; this primer focuses on “how to use the tools correctly.”

### Before you start: what you should always do

1) **Plot the series in levels** (look for trends, breaks).
2) **Choose transformations** (diff/logdiff) for stationarity.
3) **Drop missing values** before tests/models.

### Stationarity tests (ADF / KPSS)

Two common tests:
- **ADF**: null = unit root (nonstationary)
- **KPSS**: null = stationary

```python
from statsmodels.tsa.stattools import adfuller, kpss

x = df["SERIES"].dropna().to_numpy()

adf_stat, adf_p, *_ = adfuller(x)
kpss_stat, kpss_p, *_ = kpss(x, regression="c", nlags="auto")

print("ADF p:", adf_p, "KPSS p:", kpss_p)
```

**Expected output / sanity check**
- trending level series often: ADF p not small, KPSS p small
- differenced series often: ADF p small, KPSS p not small

### Cointegration (Engle–Granger test)

If two series are individually nonstationary but move together long-run, they may be cointegrated.

```python
from statsmodels.tsa.stattools import coint

y = df["Y"].dropna()
x = df["X"].dropna()

stat, p, _ = coint(y, x)
print("coint p:", p)
```

### VAR (vector autoregression)

VAR models multiple stationary-ish series jointly.

```python
from statsmodels.tsa.api import VAR

X = df[["UNRATE", "FEDFUNDS", "INDPRO"]].astype(float).dropna()
X = X.diff().dropna()  # common stationarity transform

model = VAR(X)
res = model.fit(maxlags=8, ic="aic")  # or choose lags manually
print("lags chosen:", res.k_ar)
print(res.summary())
```

**Expected output / sanity check**
- `res.k_ar` is the chosen lag length
- `res.is_stable(verbose=False)` should be True for a stable VAR

### Granger causality (predictive, not causal)

```python
res.test_causality("UNRATE", ["FEDFUNDS"]).summary()
```

Interpretation: “do lagged FEDFUNDS help predict UNRATE beyond lagged UNRATE?”

### Impulse responses (IRFs)

```python
irf = res.irf(12)
irf.plot(orth=True)  # orthogonalized IRFs (ordering matters)
```

**Important:** orthogonalized IRFs depend on a Cholesky ordering.

### Common pitfalls (and quick fixes)

- **Nonstationary inputs:** VAR on levels can be nonsense.
  - Fix: difference/logdiff; or use cointegration/VECM logic.
- **Too many lags:** eats degrees of freedom and can destabilize the model.
  - Fix: try smaller maxlags, compare AIC/BIC, check diagnostics.
- **Misinterpreting Granger causality:** it is about predictive content, not structural causality.
- **Forgetting ordering:** orth IRFs change when you reorder variables.


<a id="construct-cointegrated-pair"></a>
## Construct cointegrated pair

### Background
Cointegration is the key exception to the “levels regressions are spurious” warning.
Two series can be nonstationary individually but have a stable long-run relationship.

### What you should see
- `x` and `y` trend over time.
- `y - x` should look roughly stationary (because we simulated cointegration).

### Interpretation prompts
- In one sentence: what does it mean for two series to be cointegrated?
- Why do we simulate first before applying to real macro series?

### Goal
Construct a pair of series that are individually nonstationary but cointegrated.



### Your Turn: Simulate a cointegrated pair


In [ ]:
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
n = 240
idx = pd.date_range('2000-01-31', periods=n, freq='ME')

x = rng.normal(size=n).cumsum()
y = 1.0 * x + rng.normal(scale=0.5, size=n)

df = pd.DataFrame({'x': x, 'y': y}, index=idx)
df.head()



<a id="engle-granger-test"></a>
## Engle-Granger test

### Background
The Engle–Granger approach:
1) regress $y$ on $x$ in levels,
2) test whether the residual is stationary.

### What you should see
- a cointegration test p-value (often small in this simulated example).

### Interpretation prompts
- What is the null hypothesis in the cointegration test?
- If the p-value were large, what would that suggest?

### Goal
Run a cointegration test and interpret the p-value carefully.



### Your Turn: Cointegration test


In [ ]:
from statsmodels.tsa.stattools import coint

t_stat, p_val, _ = coint(df['y'], df['x'])
{'t': t_stat, 'p': p_val}



<a id="error-correction-model"></a>
## Error correction model

### Background
An ECM links:
- short-run changes ($\Delta y_t$)
- to long-run disequilibrium (lagged residual from the levels relationship).

### What you should see
- an estimated coefficient on `u_lag1` (often negative in a stable cointegrated system).
- interpretation as “speed of adjustment.”

### Interpretation prompts
- What does the sign of the error-correction coefficient mean?
- Why do we use the lagged residual rather than the current residual?

### Goal
Fit an ECM:
- short-run changes depend on long-run disequilibrium (lagged residual)



### Your Turn: Fit ECM


In [ ]:
import statsmodels.api as sm

# Long-run regression
lr = sm.OLS(df['y'], sm.add_constant(df[['x']], has_constant='add')).fit()
df['u'] = lr.resid

# ECM regression
ecm = pd.DataFrame({
    'dy': df['y'].diff(),
    'dx': df['x'].diff(),
    'u_lag1': df['u'].shift(1),
}).dropna()

res = sm.OLS(ecm['dy'], sm.add_constant(ecm[['dx', 'u_lag1']], has_constant='add')).fit()
res.params



<a id="interpretation"></a>
## Interpretation

Write 5-8 sentences:
- What does the error-correction coefficient mean?
- What would you expect if there were no cointegration?



<a id="checkpoint-self-check"></a>
## Checkpoint (Self-Check)
Run a few asserts and write 2-3 sentences summarizing what you verified.



In [ ]:
import pandas as pd

# TODO: Validate your time series table is well-formed.
# Example (adjust variable names):
# assert isinstance(df.index, pd.DatetimeIndex)
# assert df.index.is_monotonic_increasing
# assert df.shape[0] > 30
#
# TODO: If you built transformed series (diff/logdiff), confirm no future leakage.
# Hint: transformations should only use past/current values (shift/diff), never future.
...



## Extensions (Optional)
- Try one additional variant beyond the main path (different features, different split, different model).
- Write down what improved, what got worse, and your hypothesis for why.



## Reflection
- What did you assume implicitly (about timing, availability, stationarity, or costs)?
- If you had to ship this model, what would you monitor?



<a id="solutions-reference"></a>
## Solutions (Reference)

Try the TODOs first. Use these only to unblock yourself or to compare approaches.

<details><summary>Solution: Construct cointegrated pair</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_cointegration_error_correction — Construct cointegrated pair
import numpy as np
import pandas as pd

rng = np.random.default_rng(0)
n = 240
idx = pd.date_range('2000-01-31', periods=n, freq='ME')

x = rng.normal(size=n).cumsum()  # random walk
y = 1.0 * x + rng.normal(scale=0.5, size=n)  # cointegrated with x

df = pd.DataFrame({'x': x, 'y': y}, index=idx)
df.head()
```

</details>

<details><summary>Solution: Engle-Granger test</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_cointegration_error_correction — Engle-Granger test
from statsmodels.tsa.stattools import coint

t_stat, p_val, _ = coint(df['y'], df['x'])
{'t': t_stat, 'p': p_val}
```

</details>

<details><summary>Solution: Error correction model</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_cointegration_error_correction — Error correction model
import statsmodels.api as sm

# Step 1: long-run relationship
lr = sm.OLS(df['y'], sm.add_constant(df[['x']], has_constant='add')).fit()
df['u'] = lr.resid

# Step 2: ECM
ecm = pd.DataFrame({
    'dy': df['y'].diff(),
    'dx': df['x'].diff(),
    'u_lag1': df['u'].shift(1),
}).dropna()

res = sm.OLS(ecm['dy'], sm.add_constant(ecm[['dx', 'u_lag1']], has_constant='add')).fit()
res.params
```

</details>

<details><summary>Solution: Interpretation</summary>

_One possible approach. Your variable names may differ; align them with the notebook._

```python
# Reference solution for 01_cointegration_error_correction — Interpretation
# Explain what the error-correction coefficient implies about mean reversion.
```

</details>

